# Project to automate api access 
+ I added my api key as config.py file  PLease get your api key https://newsapi.org/docs/get-started 

In [1]:
import pandas as pd
import config
import math
import csv
import sys
from newsapi import NewsApiClient
import newspaper
import requests
from newspaper import fulltext
import time
# Hit Api with credentials
newsapi = NewsApiClient(api_key=config.api_key)

### Grab all sources
+ read through available sources list and make df storing domain and source name
+ I hit a ton of sources below, we can clearly narrow it down 

In [ ]:
def error_checks():
    ##validate that df shape[0] is equal to expected query count
    print("we should expect: {} articles".format(sum(total_count)))
    print("we have: {} articles".format(Bernie_df.shape[0]))


    ## test 100 results theory
    over_100 = sum([x-100 for x in total_count if x>100 ])
    print("{} results came into query that exceeded 100 hits in a day.".format(over_100))


    ##unique links
    unique_array = Bernie_df.url.unique()
    print("we have {} unique links".format(unique_array.shape[0]))
    
    duplicateRowsDF = Bernie_df[Bernie_df.duplicated("url")]
    print("we have duplciate shape df of: {} ".format(duplicateRowsDF.shape))

def clean_query(query):
    for x in query['articles']:
        try:
            x["source"] = x["source"]["name"]
        except:
            pass
        try:
            x['publishedAt'] = str.split(x['publishedAt'], "T")[0]
        except:
            pass
        try:
            del x['urlToImage']
        except KeyError:
            pass
    my_df = pd.DataFrame(query["articles"])
    return my_df

def hit_api(start, end, q, myString2):

    # catch bug with formatted strings for dates
    if end < 10:
        start_str = "0" + str(start)
        end_str = "0" + str(end)
    elif end == 10:
        start_str = "0" + str(start)
        end_str = str(end)
    else:
        start_str = str(start)
        end_str = str(end)

    # API query
    all_articles = newsapi.get_everything(q=q,
                                          sources=myString2,
                                          language='en',
                                          from_param='2019-11-{}'.format(
                                              start_str),
                                          to='2019-11-{}'.format(end_str),
                                          sort_by='relevancy',
                                          page_size=100,
                                          page=1)

    # get count
    total_pages = math.ceil(all_articles["totalResults"]/100)
    print("query will return: " + str(all_articles["totalResults"]))

    # store count to check versus dimension of df later
    total_count.append(all_articles["totalResults"])

    # Clen query
    all_articles = clean_query(all_articles)

    # append to list
    candidates_list.append(all_articles)
    return(candidates_list)


### Choosing data sources
+ Lets attempt to grab some sources from different geographic locations as well as different idological perspectives

+ categorizing news sources
    + Traditional TV MSM
        +  http://us.cnn.com   
        +  http://www.cnbc.com 
        +  http://www.foxnews.com  
        +  http://www.msnbc.com  
        +  https://abcnews.go.com  
        +  http://www.nbcnews.com  
    + Traditional publications 
        +  http://www.nytimes.com  
        +  https://www.washingtonpost.com 
        
    + Internet Sources
        +  http://www.huffingtonpost.com 
        +  https://www.politico.com
        +  http://www.breitbart.com 
        +  https://news.google.com 
        +  https://www.buzzfeed.com 
        +  https://news.vice.com  
    + Financial publications
        +  http://www.economist.com
        +  http://www.bloomberg.com 
        +  http://www.businessinsider.com 
        +  http://www.wsj.com
        +  http://fortune.com  
        
    + News aggregators
        +  https://apnews.com/ 
        +  http://www.reuters.com 
    + foreign reporting
         + http://www.aljazeera.com  
         + http://www.bbc.co.uk/news   
         + https://www.jpost.com/  
         + http://timesofindia.indiatimes.com 
         + https://russian.rt.com 
         + https://www.theguardian.com/uk 
         + http://www.independent.co.uk  
         + http://www.telegraph.co.uk  


 
### Build Query Strings

In [30]:

## import data dictionary form github
url = 'https://raw.githubusercontent.com/mburke65/Data_698_Final/master/lists.csv'
df = pd.read_csv(url, error_bad_lines=False)

# picking data sources from file on github
wapo__nym = df.iloc[[124,87],:]
wsj__NYT = df.iloc[[ 123, 117],:]
list_sources = wapo__nym["sources"].tolist()

# build out string for query request
myString = ",".join(list_sources)

list_sources2 = wsj__NYT["sources"].tolist()
myString2 = ",".join(list_sources2)


'the-wall-street-journal,the-new-york-times'

## Now lets begin process of automating query calls
+ first lets build function to clean query returns

In [4]:
def clean_query(query):
    for x in query['articles']:
        try:
            x["source"] = x["source"]["name"]
        except:
            pass
        try:
            x['publishedAt'] = str.split(x['publishedAt'], "T")[0]
        except:
            pass
        try:
            del x['urlToImage']
        except KeyError:
            pass
    my_df = pd.DataFrame(query["articles"])
    return my_df

## Function to hit the api


In [7]:

candidates_list = []
total_count = []

# Make first call
def hit_api(start, end, q, myString2):

    # catch bug with formatted strings for dates
    if end < 10:
        start_str = "0" + str(start)
        end_str = "0" + str(end)
    elif end == 10:
        start_str = "0" + str(start)
        end_str = str(end)
    else:
        start_str = str(start)
        end_str = str(end)

    # API query
    all_articles = newsapi.get_everything(q=q,
                                          sources=myString2,
                                          language='en',
                                          from_param='2019-11-{}'.format(
                                              start_str),
                                          to='2019-11-{}'.format(end_str),
                                          sort_by='relevancy',
                                          page_size=100,
                                          page=1)

    # get count
    total_pages = math.ceil(all_articles["totalResults"]/100)
    print("query will return: " + str(all_articles["totalResults"]))

    # store count to check versus dimension of df later
    total_count.append(all_articles["totalResults"])

    # Clen query
    all_articles = clean_query(all_articles)

    # append to list
    candidates_list.append(all_articles)
    return(candidates_list)

# Built out a loop (Important
+ Simple, look at first day of September to last day incrementing start and end by 1 each time
+ We need to expand on queires here, simply just change bernie or bernie sanders to Warren or Elizabeth warren, Biden OR Joe Biden etc



In [8]:
for x in range(20, 21):
    df = hit_api(x, x+1, 'Bernie OR Bernie Sanders', myString2)

# collapse list on itself to build big df
Bernie_df = pd.concat(df)
Bernie_df = Bernie_df.reset_index(drop=True)

query will return: 12


## Data sanity checks

### Issues
+ The counts don't match up between expected and actual because the query results are limited to first 100 hits
    + Won't be an issue with less publications, but we need to be careful splitting candidate calls up.
+ Not all links are unique
    + This is expected given internet sources reposting articles from other sources

In [9]:


error_checks()

we should expect: 12 articles
we have: 12 articles
0 results came into query that exceeded 100 hits in a day.
we have 12 unique links
we have duplciate shape df of: (0, 7) 


## Display duplicates

### Hitting newspaper 3k Api with links
+ Count print showing amount articles left
+ No words found prints it article has no words
+ headers needs to be custom changed to your own headers
    + click here for your useragent header  https://www.whatismybrowser.com/detect/what-is-my-user-agent
+ I added error check to catch videos which populates the article with a statement that just reads 'no words found'


In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}
## Build counts as check on progress of loop. will count down Url left to hit
counts=Bernie_df['url'].shape[0]
list_full_text=[]
for link in Bernie_df['url']:
    counts-=1
    print(counts)
    html = requests.get(link,headers=headers).text
    try: text = fulltext(html)
    except: 
        print("no words found")
        text="no words found"
        list_full_text.append(text)
        continue
    list_full_text.append(text)
Bernie_df['full_art']=pd.Series(list_full_text)
Bernie_df.to_csv('data1'+str(time.time+'.csv')

Bernie_df["url"]

In [ ]:
Bernie_df['full_art']=pd.Series(list_full_text)

In [ ]:
Bernie_df.to_csv('data1'+str(time.time+'.csv')

## Conclusion
+ we need to functionize streamline and clean up query calls.
+ sorry my python is rusty.